In [1]:
import os
import discord
from discord.ext import commands
from discord import Intents, Client, Message
from random import choice, randint
import pandas as pd

In [2]:
from sqlalchemy import create_engine

username = 'OliviaJYang'
password = 'springds2024'
host = '127.0.0.1'
port = '3306'  
database_name = 'rentaldb'

#create SQLAlchemy engine object to connect to database
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}") 

import pandas as pd

query = "SELECT * FROM PropertyDetails"
prop_det_df = pd.read_sql(query, con=engine)
prop_det_df.head()


,property_id,residential_id,estimated_value,sale_price,property_tax_rate
0,21973,1,11830.0,206000.0,1.02595
1,21974,2,42420.0,52000.0,1.02595
2,21975,1,11830.0,206000.0,1.02595
3,21976,3,47040.0,256000.0,1.02595
4,21977,1,11830.0,206000.0,1.02595


In [3]:
this = prop_det_df.loc[prop_det_df['property_id'] == 21973,'property_tax_rate']
this[0]

1.02595

In [4]:
# Your bot's token obtained from the Discord Developer Portal
TOKEN = 'TOKEN HERE'
CHANNEL_ID = 1237890312937148527

# Initialize the Discord client and bot
intents: intents = Intents.default()
intents.message_content = True
client = Client(intents = intents)
client.token = TOKEN

bot = commands.Bot(command_prefix = "", intents = discord.Intents.all())

i=0
j=0
k=0

In [5]:
def get_response(user_input: str) -> str:
    lowered: str = user_input.lower()
    global i
    global j
    global k

    if lowered == '':
        return 'Well, you are quiet'
    elif i>0:
        rand_prop = choice(prop_det_df['property_id'])
        ans = prop_det_df.loc[prop_det_df['property_id'] == rand_prop,'residential_id']
        # Check if the user's answer is correct
        true_ans = prop_det_df.loc[ans.index[0],'residential_id']
        if int(lowered) == int(true_ans):
            i-= 1
            return 'Correct! Well done.'
        else:
            i-= 1
            return f'Incorrect! The correct answer is: {true_ans}'
    elif j>0:
         # output correct price
        prop_num = int(lowered)
        j-= 1
        resp = prop_det_df.loc[prop_det_df['property_id'] == prop_num,'sale_price']
        true_resp = prop_det_df.loc[resp.index[0],'sale_price']
        return str(true_resp)
    elif k>0:
        # output correct prop tax
        prop_num = int(lowered)
        k-= 1
        resp = prop_det_df.loc[prop_det_df['property_id'] == prop_num,'property_tax_rate']
        true_resp = prop_det_df.loc[resp.index[0],'property_tax_rate']
        return str(true_resp)
    elif 'help' in lowered:
        return '''Welcome to the Property Bot! 
            Use game to start a game where you guess what a random house's residential id is ranging from 1-5 
            with 1-4 indicating how many separate living spaces it's split into and 5 being none
            
            Use price to see the sale price of a house 
            given its property id ranging from 21973 - 26814
            
            Use proptaxrate tp see the property tax rate of a house 
            given its property id ranging from 21973 - 26814'''
    elif 'game' in lowered:
        i+=1
        return "Guess the random house's residential id 1,2,3,4, or 5"
    elif 'price' in lowered:
        j+=1
        return 'Enter property id ranging from 21973 - 26814 for its price'
    elif 'proptaxrate' in lowered:
        k+=1
        return 'Enter property id ranging from 21973 - 26814 for its property tax rate'
    else:
        return "Incoherrent input, type help for suggestions"
        

In [6]:
# Message Function
@client.event
async def send_message(message: Message, user_message:str):
    if not user_message:
        print('Message was empty because intents were not enabled...prob')
        return
    if is_private := user_message[0] == '?':
        user_message = user_message[1:]

    try:
        response: str = get_response(user_message)
        await message.author.send(response) if is_private else await message.channel.send(response)
    except Exception as e:
        print(e)
    

In [7]:
#Handle start up for bot
@client.event
async def on_ready():
    print(f'{client.user} has connected to Discord!')
    channel = bot.get_channel(CHANNEL_ID)


In [8]:
#Handle Messages
@client.event
async def on_message(message: Message):
    #bot talking to itself case and checking for previous bot statement for back and forth
    if message.author == client.user:
        return


    username: str=str(message.author)
    user_message:str = message.content
    channel: str = str(message.channel)

    print(f'[{channel}], {username}: "{user_message}"')
    await send_message(message, user_message)

        

In [9]:
@client.event
async def shutdown(context):
    exit()

In [10]:
# Run the bot
def main() -> None:
    client.run(token = TOKEN)

In [11]:
import asyncio
asyncio.get_event_loop().create_task(client.start(TOKEN))

<Task pending name='Task-5' coro=<Client.start() running at C:\Users\olivi\AppData\Local\Programs\Python\Python312\Lib\site-packages\discord\client.py:756>>

Property Bot#2662 has connected to Discord!
[bot-channel], .oliviayang: "game"
[bot-channel], .oliviayang: "2"
[bot-channel], .oliviayang: "help"
[bot-channel], .oliviayang: "game"
[bot-channel], .oliviayang: "2"
[bot-channel], .oliviayang: "price"
[bot-channel], .oliviayang: "21974"
[bot-channel], .oliviayang: "proptaxrate"
[bot-channel], .oliviayang: "21974"
